# Notification Example

Notification is a non-modal option to show errors or other important information without interrupting the user's workflow. The [IntelliJ Platform Plugin SDK](https://plugins.jetbrains.com/docs/intellij/notifications.html) provides more detailed description of the capabilities and UI guidelines.

Let's create a simple notification. To do that, let's enable IntelliJ Platform Integration and load necessary libraries.

In [ ]:
%useLatestDescriptors
%use intellij-platform

Run this cell and make sure that the integration was properly initialized. Before doing this, choose _Run in IDE Process_ mode.

## Creating a Simple Notification Baloon

Let's create a simple notification balloon.

In [ ]:
import com.intellij.notification.NotificationGroupManager
import com.intellij.notification.NotificationType
import com.intellij.notification.NotificationsManager

runInEdt {
    NotificationGroupManager.getInstance()
        .getNotificationGroup("Kotlin Notebook session info")
        .createNotification("Hello from Kotlin Notebook", NotificationType.INFORMATION)
        .notify(currentProject())
}

This will show a *timed notification* that remains on the screen for 10 seconds and then disappear.

Each notification belongs to a *notification group*. In this example, we will reuse the notification group provided by the Kotlin Notebook.

We will provide the notification text and treat it as a general innformation.

# Creating a Notification in the Custom Notification Group

A custom notification group allows the user to control how each notification is displayed.
This can be customized in *Appearance & Behavior | Notifications*.

In plugin descriptor, the notification group is specified in the `id` attribute.

```
<extensions defaultExtensionNs="com.intellij">
  <notificationGroup id="Kotlin Notebook Sample Notifications"
                     displayType="BALLOON" />
</extensions>
```

In Kotlin Notebook, we can specify the notification group by using the corresponding extension point class.
In this example, it is the [`NotificationGroupEP`](https://github.com/JetBrains/intellij-community/blob/master/platform/ide-core-impl/src/com/intellij/notification/impl/NotificationGroupEP.java) class.

### Working Around Platform SDK Issues
Due to [IJPL-213568](https://youtrack.jetbrains.com/issue/IJPL-213568), we cannot set the display type of the notification group programmaticaly.
Until this is fixed, we will resort to the reflection.

In [ ]:
import com.intellij.notification.impl.NotificationGroupEP

fun NotificationGroupEP.setDisplayType(displayTypeValue: String) {
    try {
        val field = this::class.java.getDeclaredField("displayType")
        field.isAccessible = true
        require(field.type.isEnum) { "Field 'displayType' is not an enum" }
        field.type.enumConstants
            .find { (it as Enum<*>).name == displayTypeValue }
            .let {
                field.set(this, it)
            }
        field.set(this, displayType)
    } catch (e: Exception) {
        error("Failed to set displayType: ${e.message}")
    }
}

## Creating a Sticky Timeline Notification

Let's create a sticky [timeline notification](https://plugins.jetbrains.com/docs/intellij/balloon.html#suggestion-and-timeline-notifications). Unlike _timed notifications_, a _sticky notification_ will not disappear after 10 seconds.

In [ ]:
import com.intellij.notification.Notification
import com.intellij.notification.NotificationAction
import com.intellij.notification.NotificationGroup
import com.intellij.notification.NotificationGroupManager
import com.intellij.notification.NotificationType
import com.intellij.notification.impl.NotificationGroupEP
import com.intellij.openapi.actionSystem.AnActionEvent
import com.intellij.openapi.extensions.ExtensionPointName
import com.intellij.openapi.ui.Messages

import java.lang.reflect.Field
import java.time.LocalDate
import java.time.format.TextStyle
import java.util.Locale

runInEdt {
    val notificationGroupEpName: ExtensionPointName<NotificationGroupEP> = ExtensionPointName.create("com.intellij.notificationGroup")
    val notificationGroupId = "Kotlin Notebook Sample Notifications"
    val notificationGroupEP = NotificationGroupEP().apply {
        id = notificationGroupId
        setPluginDescriptor(notebookPluginDescriptor)
        setDisplayType("STICKY_BALLOON")
    }

    registerExtension(notificationGroupEpName, notificationGroupEP)

    val dayOfWeek = LocalDate.now().dayOfWeek.getDisplayName(TextStyle.FULL, Locale.ENGLISH)

    NotificationGroupManager.getInstance()
        .getNotificationGroup(notificationGroupId)
        .createNotification("It's $dayOfWeek.", NotificationType.INFORMATION)
        .addAction(NotificationAction.createExpiring("See Wikipedia") { _, _ ->
            BrowserUtil.browse("https://en.wikipedia.org/wiki/$dayOfWeek")
        })
        .notify(currentProject())
}

First, we have created an extension point with a corresponding name: `com.intellij.notificationGroup`.
Then, we created an instance of `NotificationGroupEP` endpoint that matches the declaration in the plugin descriptor. Then, the notification group identifier (`id`) is set.

In the programmatic approach, we need to set the reference to the plugin descriptor that is internally maintained by the Kotlin Notebook (`notebookPluginDescriptor)`.

By using our helper function, we declare this notification group to be _sticky_ by using the `STICKY_BALOON` string constant.

The notification can contain arbitrary number of actions, presented as hypertext links. We can use `addAction` method, along with `NotificationAction` factory methods, to handle such user interactions.

# Create a Suggestion Notification

A [suggestion notification](https://plugins.jetbrains.com/docs/intellij/balloon.html#suggestion-and-timeline-notifications) shows the recommended action as a noticeable button. By default, it is implemented as a sticky notification.

In [ ]:
import com.intellij.ide.BrowserUtil
import com.intellij.notification.Notification
import com.intellij.notification.NotificationAction
import com.intellij.notification.NotificationGroup
import com.intellij.notification.NotificationGroupManager
import com.intellij.notification.NotificationType
import com.intellij.notification.impl.NotificationGroupEP
import com.intellij.openapi.actionSystem.AnActionEvent
import com.intellij.openapi.extensions.ExtensionPointName
import com.intellij.openapi.ui.Messages

import java.lang.reflect.Field
import java.time.LocalDate
import java.time.format.TextStyle
import java.util.Locale

runInEdt {
    val notificationGroupEpName: ExtensionPointName<NotificationGroupEP> = ExtensionPointName.create("com.intellij.notificationGroup")
    val notificationGroupId = "Kotlin Notebook Sample Suggestions"
    val notificationGroupEP = NotificationGroupEP().apply {
        id = notificationGroupId
        setDisplayType("STICKY_BALLOON")
        setPluginDescriptor(notebookPluginDescriptor)
    }

    registerExtension(notificationGroupEpName, notificationGroupEP)

    val dayOfWeek = LocalDate.now().dayOfWeek.getDisplayName(TextStyle.FULL, Locale.ENGLISH)

    NotificationGroupManager.getInstance()
        .getNotificationGroup(notificationGroupId)
        .createNotification("It's $dayOfWeek.", NotificationType.INFORMATION)
        .addAction(NotificationAction.createSimpleExpiring("Accept") {})
        .addAction(NotificationAction.create("See Wikipedia entry") { _: AnActionEvent, n: Notification ->
            BrowserUtil.browse("https://en.wikipedia.org/wiki/$dayOfWeek")
            n.expire()
        })
        .setSuggestionType(true)
        .notify(currentProject())
}

The `setSuggestionType` method with `true` parameter configures this notification as a _suggestion_. The first action is automatically displayed as a button, while the rest is shown in hyperlinks.

### Handling Notification Actions

The `addAction` method will add a new `AnAction` to the notification. To simplify the configuration, a `NotificationAction` class has multiple helper methods to create notification-specific actions along with implicit behaviour and Kotlin-friendly API.

In the example, we use two helper methods. The `createSimpleExpiring` method will create a simple action that will execute a given handler and then automatically expire the notification including its removal from the Notifications tool window.

The `create` method to create a simple action along with action handler that opens a default browser with a Wikipedia page. On top of that, the notification will be explicitly expired by calling the `expire` method on the notification object provided in the handler parameter.

# Summary

* Create `NotificationGroupEP` extension point to configure notifications and register it in the Kotlin Notebook.
* Create helper function that overcomes Platform issue with programmatic registration of notification group endpoints.
* Use `NotificationGroupManager` to create notification instances in the specific notification group.
* Use `STICKY_BALLOON` to configure sticky notifications.
* Use `setSuggestionType` to show suggestion notifications with prominent button as a first action
* Use `addAction` along with `NotificationActions` to configure button or hyperlink handlers.